# Anthropic Example

First let's install the required packages

In [ ]:
%pip install -U anthropic
%pip install -U agentops

Then import them

In [ ]:
from anthropic import Anthropic, AsyncAnthropic
import agentops
import os
from dotenv import load_dotenv

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "<your_anthropic_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["anthropic-example"])

## Sync Example
We will demonstrate a basic sync call to Anthropic using the Claude 3.5 Sonnet model.

In [ ]:
client = Anthropic()

In [ ]:
stream = client.messages.create(
    max_tokens=1024,
    model="claude-3-5-sonnet-20240620",
    messages=[
        {
            "role": "user",
            "content": "How much wood could a wood chuck chuck if a wood chuck could chuck wood?",
        }
    ],
    stream=True,
)

response = ""
for event in stream:
    if event.type == "content_block_delta":
        response += event.delta.text
    elif event.type == "message_stop":
        print(response)

## Async Example
The async example is very similar to the sync example, but it uses the AsyncAnthropic client from the anthropic library.

In [ ]:
aclient = AsyncAnthropic()

In [ ]:
async def main() -> None:
    message = await aclient.messages.create(
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": "Explain everything you know about the Leibniz Equation.",
            }
        ],
        model="claude-3-5-sonnet-20240620",
    )
    print(message.content[0].text)


await main()

## Tool Calling Example
Anthropic models support tool calling, which allows the model to call external APIs.

In [ ]:
stream = client.messages.create(
    max_tokens=1024,
    model="claude-3-5-sonnet-20240620",
    tools=[
        {
            "name": "web_search",
            "description": "Search the web for information",
            "input_schema": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "The search query"}
                },
                "required": ["query"],
            },
        }
    ],
    messages=[
        {
            "role": "user",
            "content": "Tell me everything you can about AgentOps",
        }
    ],
    stream=True,
)

response = ""
for event in stream:
    if event.type == "content_block_delta":
        if event.delta.type == "text":
            response += event.delta.text
    elif event.type == "message_stop":
        print(response)

In [ ]:
agentops.end_session("Success")